In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

In [2]:
df = pd.read_pickle('/content/drive/My Drive/capstone/updated_processed.pkl')
df.head()

,Additional_Number_of_Scoring,Average_Score,Review_Total_Negative_Word_Counts,Total_Number_of_Reviews,Review_Total_Positive_Word_Counts,Total_Number_of_Reviews_Reviewer_Has_Given,Reviewer_Score,reviews,Hotel_Country,review_month,Trip,Stayed nights,yr,test_reviews,test_positive_review,test_negative_review,reviews_processed,positive_reviews_processed,negative_reviews_processed,Combined_pos,Combined_neg,Combined_neu
0,241,8.2,138,2500,48,8,7.5,Bathtub and sink flush wasn t working properl...,4,8,1,6,2016,[bathtub and sink flush was not t working prop...,[we ve had a room with sea view and was lovely...,[bathtub and sink flush was not t working prop...,bathtub sink flush not working properly even t...,room sea view lovely location great want go be...,bathtub sink flush not working properly even t...,0.155,0.024,0.821
1,493,8.6,23,2865,9,3,5.4,The bedroom was tight and very small They exp...,3,12,0,2,2016,[the bedroom was tight and very small they exp...,[to be near to the main train station],[the bedroom was tight and very small they exp...,bedroom tight small explain hotel built old fa...,near main train station,bedroom tight small explain hotel built old fa...,0.000,0.000,1.000
2,908,8.5,0,3980,13,8,9.2,Great to have ipads in the room and the break...,5,2,1,1,2016,[great to have ipads in the room and the break...,[great to have ipads in the room and the break...,[no negative],great ipads room breakfast amazing,great ipads room breakfast amazing,negative,0.725,0.000,0.275
3,117,8.3,47,694,79,1,9.2,There was only one small issue with the room ...,1,7,1,6,2017,[there was only one small issue with the room ...,[the location was excellent it was off the cha...,[there was only one small issue with the room ...,one small issue room coffee machine come stand...,location excellent champ elysees minute walk a...,one small issue room coffee machine come stand...,0.405,0.000,0.595
4,300,9.0,68,1254,25,1,9.6,Price of breakfast was high We went to an ama...,5,3,1,1,2016,[price of breakfast was high we went to an ama...,[amazing location and quality of hotel being c...,[price of breakfast was high we went to an ama...,price breakfast high went amazing cafe road hi...,amazing location quality hotel close tube stat...,price breakfast high went amazing cafe road hi...,0.342,0.077,0.581


In [3]:
import nltk
nltk.download('stopwords')
nltk.download('wordnet')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


True

In [4]:
from nltk.stem import WordNetLemmatizer
from nltk.corpus import stopwords

In [5]:
import nltk
nltk.download('vader_lexicon')
from nltk.sentiment.vader import SentimentIntensityAnalyzer
sid = SentimentIntensityAnalyzer()
from sklearn.feature_extraction.text import TfidfVectorizer

[nltk_data] Downloading package vader_lexicon to /root/nltk_data...
[nltk_data]   Package vader_lexicon is already up-to-date!


/usr/local/lib/python3.6/dist-packages/nltk/twitter/__init__.py:20: UserWarning: The twython library has not been installed. Some functionality from the twitter package will not be available.
  warnings.warn("The twython library has not been installed. "


In [6]:
stopwords_l = stopwords.words('english')
stopwords_l.remove('not')

In [7]:
from sklearn.preprocessing import StandardScaler
from sklearn.ensemble import BaggingRegressor,RandomForestRegressor,AdaBoostRegressor
from sklearn import model_selection
from sklearn import metrics
from sklearn.preprocessing import StandardScaler
from sklearn.impute import KNNImputer
from sklearn.tree import DecisionTreeClassifier, DecisionTreeRegressor
from sklearn.model_selection import GridSearchCV
from sklearn.pipeline import Pipeline
# from missingpy import KNNImputer

In [ ]:
# X : combined revies and y = Combined_pos,Combined_neg

X = df['reviews_processed']
y = df['Combined_pos']
y1=df['Combined_neg']

In [ ]:
# #sub model1: where target variable is 'Combined_pos'

# param ={'model__max_depth':np.arange(1,100)}
# pipeline = Pipeline([
#     ('tfidf', TfidfVectorizer(analyzer='word', min_df=0,stop_words= stopwords_l)),
#     ('model',DecisionTreeRegressor()),
#     ])
# grid = GridSearchCV(pipeline, param_grid= param,cv=3,scoring='neg_mean_squared_error')
# grid.fit(X,y)
# grid.best_params_

In [ ]:
# got the best params by gridsearchcv i.e. 78 as max_depth
from sklearn.pipeline import Pipeline
#Sub model 1: where target variable : 'Combined_pos'
model1 = Pipeline([('tfidf', TfidfVectorizer(analyzer='word', min_df=0,stop_words= stopwords_l)),
                     ('dt', DecisionTreeRegressor(max_depth=78)),
])
model1.fit(X,y)
scores = model_selection.cross_val_score(model1, X, y, cv= 3, scoring='neg_mean_squared_error')
rmse = np.sqrt(np.abs(scores))
print('bias_error',np.mean(rmse))
print('var_error',np.std(rmse,ddof=1))

bias_error 0.1472825924564989
var_error 0.001427810621754036


In [ ]:
#Sub model 2: where target variable : 'Combined_neg'
# got the best params by gridsearchcv i.e. 23 as max_depth
model2 =  Pipeline([('tfidf', TfidfVectorizer(analyzer='word', min_df=0,stop_words= stopwords_l)),
                     ('dt', DecisionTreeRegressor(max_depth= 23)),
])
model2.fit(X,y1)
scores = model_selection.cross_val_score(model2, X, y1, cv= 3, scoring='neg_mean_squared_error')
rmse = np.sqrt(np.abs(scores))
print('bias_error',np.mean(rmse))
print('var_error',np.std(rmse,ddof=1))

bias_error 0.09808139329796
var_error 0.00044064773842754314


In [ ]:
df['Combined_avg_polarity'] = (y - y1)/2


In [ ]:
df[df['Combined_avg_polarity'] <= 0].Combined_avg_polarity.count(),df[df['Combined_avg_polarity'] > 0].Combined_avg_polarity.count()

(13148, 36529)

In [ ]:
13148+ 36529

49677

In [ ]:
df.head(2)

,Additional_Number_of_Scoring,Average_Score,Review_Total_Negative_Word_Counts,Total_Number_of_Reviews,Review_Total_Positive_Word_Counts,Total_Number_of_Reviews_Reviewer_Has_Given,Reviewer_Score,reviews,Hotel_Country,review_month,Trip,Stayed nights,yr,test_reviews,test_positive_review,test_negative_review,reviews_processed,positive_reviews_processed,negative_reviews_processed,Combined_pos,Combined_neg,Combined_neu,Combined_avg_polarity
0,241,8.2,138,2500,48,8,7.5,Bathtub and sink flush wasn t working properl...,4,8,1,6,2016,[bathtub and sink flush was not t working prop...,[we ve had a room with sea view and was lovely...,[bathtub and sink flush was not t working prop...,bathtub sink flush not working properly even t...,room sea view lovely location great want go be...,bathtub sink flush not working properly even t...,0.155,0.024,0.821,0.0655
1,493,8.6,23,2865,9,3,5.4,The bedroom was tight and very small They exp...,3,12,0,2,2016,[the bedroom was tight and very small they exp...,[to be near to the main train station],[the bedroom was tight and very small they exp...,bedroom tight small explain hotel built old fa...,near main train station,bedroom tight small explain hotel built old fa...,0.000,0.000,1.000,0.0000


# Building the models for Positive reviews

In [ ]:
positive_reviews = pd.DataFrame()
positive_reviews['positive_reviews_processed'] = df['positive_reviews_processed']

positive_reviews['Positive_pos'] = positive_reviews['positive_reviews_processed'].apply(lambda x:sid.polarity_scores(x)['pos'])
positive_reviews['Positive_neg'] = positive_reviews['positive_reviews_processed'].apply(lambda x:sid.polarity_scores(x)['neg'])
positive_reviews['Positive_neu'] = positive_reviews['positive_reviews_processed'].apply(lambda x:sid.polarity_scores(x)['neu'])
positive_reviews

,positive_reviews_processed,Positive_pos,Positive_neg,Positive_neu
0,room sea view lovely location great want go be...,0.305,0.0,0.695
1,near main train station,0.000,0.0,1.000
2,great ipads room breakfast amazing,0.725,0.0,0.275
3,location excellent champ elysees minute walk a...,0.484,0.0,0.516
4,amazing location quality hotel close tube stat...,0.402,0.0,0.598
...,...,...,...,...
49672,bed double family instead single bed,0.000,0.0,1.000
49673,well equipped compact room fabulous setting ov...,0.407,0.0,0.593
49674,thought hotel lacked every aspect hospitality ...,0.080,0.0,0.920
49675,lovely location st katharines dock lovely,0.655,0.0,0.345


In [ ]:
# X : positive revies and y = positive_pos,positivetive_neg

pos_X = positive_reviews['positive_reviews_processed']
pos_y = positive_reviews['Positive_pos']
pos_y1=positive_reviews['Positive_neg']

In [ ]:
sid.polarity_scores('quite good but not good')

{'compound': 0.7496, 'neg': 0.0, 'neu': 0.32, 'pos': 0.68}

In [ ]:
pos_y1[pos_y1 >= 0].count()

49677

In [ ]:
# dt_model = DecisionTreeRegressor()
# #n_neighbors ideal search range could be 15-20% of size of data
# param={'max_depth':np.arange(1,10)}
# GS=GridSearchCV(Pos_model1,param,cv=3,scoring='neg_mean_squared_error')
# GS.fit(pos_X,pos_y)
# # Pos_model1.fit()
# GS.best_params_

In [ ]:
param ={'model__max_depth':np.arange(1,100)}
pipeline = Pipeline([
#     # ('vect', CountVectorizer(min_df=40,ngram_range=(1,4))),
    ('tfidf', TfidfVectorizer(analyzer='word', min_df=0,stop_words= stopwords_l)),
    # ('grid',GridSearchCV(pipeline, param,cv=5,scoring='neg_mean_squared_error')),
  ('model',DecisionTreeRegressor()),
    # ('grid',GridSearchCV('model',param_grid= param,cv=5,scoring='neg_mean_squared_error')),
    ])
grid = GridSearchCV(pipeline, param_grid= param,cv=3,scoring='neg_mean_squared_error')
grid.fit(pos_X,pos_y)
# pipeline.named_steps['grid'].best_params_
# grid.score(pos_X,pos_y)
grid.best_params_

{'model__max_depth': 99}

In [ ]:
#Sub model 1: where target variable : 'Positive_pos'
Pos_model1 =  Pipeline([('tfidf', TfidfVectorizer(analyzer='word', min_df=0,stop_words= stopwords_l)),
                     ('dt', DecisionTreeRegressor(max_depth= 99)),
])
Pos_model1.fit(pos_X,pos_y)
scores = model_selection.cross_val_score(Pos_model1, pos_X, pos_y, cv= 3, scoring='neg_mean_squared_error')
rmse = np.sqrt(np.abs(scores))
print('bias_error',np.mean(rmse))
print('var_error',np.std(rmse,ddof=1))


bias_error 0.12772609947310787
var_error 0.0006189058386366537


In [ ]:
#Sub model 2: where target variable : 'Positive_neg'
Pos_model2 =  Pipeline([('tfidf', TfidfVectorizer(analyzer='word', min_df=0,stop_words= stopwords_l)),
                     ('dt', DecisionTreeRegressor(max_depth=32)),
])
Pos_model2.fit(pos_X,pos_y1)
# print('Score:',Pos_model2.score(pos_X,pos_y1))
scores = model_selection.cross_val_score(Pos_model2, pos_X,pos_y1, cv= 3, scoring='neg_mean_squared_error')
rmse = np.sqrt(np.abs(scores))
print('bias_error',np.mean(rmse))
print('var_error',np.std(rmse,ddof=1))

bias_error 0.04465911553293197
var_error 0.00171228698560155


In [ ]:
# positive      -(nega)      label(0-10)( <0: neg : >0 positive)
# 0.54          0.9      avg : abs(0.59)
# 0.24          -(-0.64)     abs(avg) : 0.2

# 0.24         0.59        compare 
# 0.0          0.0         neutral

In [ ]:
positive_reviews['avg_polarity'] = (pos_y - pos_y1)/2


In [ ]:
positive_reviews[positive_reviews['avg_polarity'] < 0].avg_polarity.count()

668

In [ ]:
positive_reviews.sample(10)

,positive_reviews_processed,Positive_pos,Positive_neg,Positive_neu,avg_polarity
38820,location wonderful,0.787,0.0,0.213,0.3935
1066,excellent staff clean,0.865,0.0,0.135,0.4325
48951,hotel good location within walking distance ha...,0.418,0.0,0.582,0.2090
6707,bed great concierge perfect breakfast copious ...,0.556,0.0,0.444,0.2780
45655,lovely crisp white bed linen super comfy bed b...,0.494,0.0,0.506,0.2470
26212,loved ability control function room ipad,0.608,0.0,0.392,0.3040
34637,nice bed,0.737,0.0,0.263,0.3685
43196,bed good lounge downstairs tv nespresso machine,0.326,0.0,0.674,0.1630
13608,close edgware station,0.000,0.0,1.000,0.0000
2647,beautiful building great staff,0.800,0.0,0.200,0.4000


In [ ]:
positive_reviews[positive_reviews['avg_polarity'] < 0].count()

positive_reviews_processed    668
Positive_pos                  668
Positive_neg                  668
Positive_neu                  668
avg_polarity                  668
dtype: int64

#For negative preprocessed reviews

In [ ]:
negative_reviews = pd.DataFrame()
negative_reviews['negative_reviews_processed'] = df['negative_reviews_processed']

negative_reviews['Negative_pos'] = negative_reviews['negative_reviews_processed'].apply(lambda x:sid.polarity_scores(x)['pos'])
negative_reviews['Negative_neg'] = negative_reviews['negative_reviews_processed'].apply(lambda x:sid.polarity_scores(x)['neg'])
negative_reviews['Negative_neu'] = negative_reviews['negative_reviews_processed'].apply(lambda x:sid.polarity_scores(x)['neu'])
negative_reviews

,negative_reviews_processed,Negative_pos,Negative_neg,Negative_neu
0,bathtub sink flush not working properly even t...,0.100,0.033,0.867
1,bedroom tight small explain hotel built old fa...,0.000,0.000,1.000
2,negative,0.000,1.000,0.000
3,one small issue room coffee machine come stand...,0.228,0.000,0.772
4,price breakfast high went amazing cafe road hi...,0.314,0.113,0.573
...,...,...,...,...
49672,negative,0.000,1.000,0.000
49673,bar area becomes late night club noticeable be...,0.089,0.067,0.844
49674,thought hotel lacked every aspect hospitality ...,0.039,0.000,0.961
49675,breakfast queue long sunday queue horrendous,0.000,0.432,0.568


In [ ]:
# # X : negative revies and y = Negative_pos,negative_neg

neg_X = negative_reviews['negative_reviews_processed']
neg_y = negative_reviews['Negative_pos']
neg_y1= negative_reviews['Negative_neg']

In [ ]:
# #sub model1: where target variable is 'Negative_pos'

param ={'model__max_depth':np.arange(1,100)}
pipeline = Pipeline([
    ('tfidf', TfidfVectorizer(analyzer='word', min_df=0,stop_words= stopwords_l)),
    ('model',DecisionTreeRegressor()),
    ])
grid = GridSearchCV(pipeline, param_grid= param,cv=3,scoring='neg_mean_squared_error')
grid.fit(neg_X,neg_y)
grid.best_params_

{'model__max_depth': 98}

In [ ]:
#Sub model 1: where target variable : 'Negative_pos'
#  with best params from the above cell (97)
Neg_model1 =  Pipeline([('tfidf', TfidfVectorizer(analyzer='word', min_df=0,stop_words= stopwords_l)),
                     ('dt', DecisionTreeRegressor(max_depth= 97)),
])
Neg_model1.fit(neg_X,neg_y)
scores = model_selection.cross_val_score(Neg_model1, neg_X, neg_y, cv= 3, scoring='neg_mean_squared_error')
rmse = np.sqrt(np.abs(scores))
print('bias_error',np.mean(rmse))
print('var_error',np.std(rmse,ddof=1))


bias_error 0.09828672493225267
var_error 0.003213997703967224


In [ ]:
param ={'model__max_depth':np.arange(1,100)}
pipeline = Pipeline([
    ('tfidf', TfidfVectorizer(analyzer='word', min_df=0,stop_words= stopwords_l)),
    ('model',DecisionTreeRegressor()),
    ])
grid = GridSearchCV(pipeline, param_grid= param,cv=3,scoring='neg_mean_squared_error')
grid.fit(neg_X,neg_y1)
grid.best_params_

{'model__max_depth': 97}

In [ ]:
# Sub model 2: where target variable : 'Negative_neg'
#  with best params from the above cell ()
Neg_model2 =  Pipeline([('tfidf', TfidfVectorizer(analyzer='word', min_df=0,stop_words= stopwords_l)),
                     ('dt', DecisionTreeRegressor(max_depth= 97)),
])
Neg_model2.fit(neg_X,neg_y)
scores = model_selection.cross_val_score(Neg_model2, neg_X, neg_y1, cv= 3, scoring='neg_mean_squared_error')
rmse = np.sqrt(np.abs(scores))
print('bias_error',np.mean(rmse))
print('var_error',np.std(rmse,ddof=1))

bias_error 0.1004367371101655
var_error 0.0015260770224528916


In [ ]:
df.head()

,Additional_Number_of_Scoring,Average_Score,Review_Total_Negative_Word_Counts,Total_Number_of_Reviews,Review_Total_Positive_Word_Counts,Total_Number_of_Reviews_Reviewer_Has_Given,Reviewer_Score,reviews,Hotel_Country,review_month,Trip,Stayed nights,yr,test_reviews,test_positive_review,test_negative_review,reviews_processed,positive_reviews_processed,negative_reviews_processed,Combined_pos,Combined_neg,Combined_neu,Combined_avg_polarity
0,241,8.2,138,2500,48,8,7.5,Bathtub and sink flush wasn t working properl...,4,8,1,6,2016,[bathtub and sink flush was not t working prop...,[we ve had a room with sea view and was lovely...,[bathtub and sink flush was not t working prop...,bathtub sink flush not working properly even t...,room sea view lovely location great want go be...,bathtub sink flush not working properly even t...,0.155,0.024,0.821,0.0655
1,493,8.6,23,2865,9,3,5.4,The bedroom was tight and very small They exp...,3,12,0,2,2016,[the bedroom was tight and very small they exp...,[to be near to the main train station],[the bedroom was tight and very small they exp...,bedroom tight small explain hotel built old fa...,near main train station,bedroom tight small explain hotel built old fa...,0.000,0.000,1.000,0.0000
2,908,8.5,0,3980,13,8,9.2,Great to have ipads in the room and the break...,5,2,1,1,2016,[great to have ipads in the room and the break...,[great to have ipads in the room and the break...,[no negative],great ipads room breakfast amazing,great ipads room breakfast amazing,negative,0.725,0.000,0.275,0.3625
3,117,8.3,47,694,79,1,9.2,There was only one small issue with the room ...,1,7,1,6,2017,[there was only one small issue with the room ...,[the location was excellent it was off the cha...,[there was only one small issue with the room ...,one small issue room coffee machine come stand...,location excellent champ elysees minute walk a...,one small issue room coffee machine come stand...,0.405,0.000,0.595,0.2025
4,300,9.0,68,1254,25,1,9.6,Price of breakfast was high We went to an ama...,5,3,1,1,2016,[price of breakfast was high we went to an ama...,[amazing location and quality of hotel being c...,[price of breakfast was high we went to an ama...,price breakfast high went amazing cafe road hi...,amazing location quality hotel close tube stat...,price breakfast high went amazing cafe road hi...,0.342,0.077,0.581,0.1325


In [ ]:
df['Positive_pos'] = positive_reviews['Positive_pos']
df['Positive_neg'] = positive_reviews['Positive_neg']

df['Negative_pos'] = negative_reviews['Negative_pos']
df['Negative_neg'] = negative_reviews['Negative_neg']


In [ ]:
df.head(2)

,Additional_Number_of_Scoring,Average_Score,Review_Total_Negative_Word_Counts,Total_Number_of_Reviews,Review_Total_Positive_Word_Counts,Total_Number_of_Reviews_Reviewer_Has_Given,Reviewer_Score,reviews,Hotel_Country,review_month,Trip,Stayed nights,yr,test_reviews,test_positive_review,test_negative_review,reviews_processed,positive_reviews_processed,negative_reviews_processed,Combined_pos,Combined_neg,Combined_neu,Combined_avg_polarity,Positive_pos,Positive_neg,Negative_pos,Negative_neg
0,241,8.2,138,2500,48,8,7.5,Bathtub and sink flush wasn t working properl...,4,8,1,6,2016,[bathtub and sink flush was not t working prop...,[we ve had a room with sea view and was lovely...,[bathtub and sink flush was not t working prop...,bathtub sink flush not working properly even t...,room sea view lovely location great want go be...,bathtub sink flush not working properly even t...,0.155,0.024,0.821,0.0655,0.305,0.0,0.1,0.033
1,493,8.6,23,2865,9,3,5.4,The bedroom was tight and very small They exp...,3,12,0,2,2016,[the bedroom was tight and very small they exp...,[to be near to the main train station],[the bedroom was tight and very small they exp...,bedroom tight small explain hotel built old fa...,near main train station,bedroom tight small explain hotel built old fa...,0.000,0.000,1.000,0.0000,0.000,0.0,0.0,0.000


In [ ]:
# 
df.to_pickle("/content/drive/My Drive/capstone/updated_processed1.pkl")

In [ ]:
df.head()

,Additional_Number_of_Scoring,Average_Score,Review_Total_Negative_Word_Counts,Total_Number_of_Reviews,Review_Total_Positive_Word_Counts,Total_Number_of_Reviews_Reviewer_Has_Given,Reviewer_Score,reviews,Hotel_Country,review_month,Trip,Stayed nights,yr,test_reviews,test_positive_review,test_negative_review,reviews_processed,positive_reviews_processed,negative_reviews_processed,Combined_pos,Combined_neg,Combined_neu
0,241,8.2,138,2500,48,8,7.5,Bathtub and sink flush wasn t working properl...,4,8,1,6,2016,[bathtub and sink flush was not t working prop...,[we ve had a room with sea view and was lovely...,[bathtub and sink flush was not t working prop...,bathtub sink flush not working properly even t...,room sea view lovely location great want go be...,bathtub sink flush not working properly even t...,0.155,0.024,0.821
1,493,8.6,23,2865,9,3,5.4,The bedroom was tight and very small They exp...,3,12,0,2,2016,[the bedroom was tight and very small they exp...,[to be near to the main train station],[the bedroom was tight and very small they exp...,bedroom tight small explain hotel built old fa...,near main train station,bedroom tight small explain hotel built old fa...,0.000,0.000,1.000
2,908,8.5,0,3980,13,8,9.2,Great to have ipads in the room and the break...,5,2,1,1,2016,[great to have ipads in the room and the break...,[great to have ipads in the room and the break...,[no negative],great ipads room breakfast amazing,great ipads room breakfast amazing,negative,0.725,0.000,0.275
3,117,8.3,47,694,79,1,9.2,There was only one small issue with the room ...,1,7,1,6,2017,[there was only one small issue with the room ...,[the location was excellent it was off the cha...,[there was only one small issue with the room ...,one small issue room coffee machine come stand...,location excellent champ elysees minute walk a...,one small issue room coffee machine come stand...,0.405,0.000,0.595
4,300,9.0,68,1254,25,1,9.6,Price of breakfast was high We went to an ama...,5,3,1,1,2016,[price of breakfast was high we went to an ama...,[amazing location and quality of hotel being c...,[price of breakfast was high we went to an ama...,price breakfast high went amazing cafe road hi...,amazing location quality hotel close tube stat...,price breakfast high went amazing cafe road hi...,0.342,0.077,0.581


In [ ]:
df.columns

Index(['Additional_Number_of_Scoring', 'Average_Score',
       'Review_Total_Negative_Word_Counts', 'Total_Number_of_Reviews',
       'Review_Total_Positive_Word_Counts',
       'Total_Number_of_Reviews_Reviewer_Has_Given', 'Reviewer_Score',
       'reviews', 'Hotel_Country', 'review_month', 'Trip', 'Stayed nights',
       'yr', 'test_reviews', 'test_positive_review', 'test_negative_review',
       'reviews_processed', 'positive_reviews_processed',
       'negative_reviews_processed', 'Combined_pos', 'Combined_neg',
       'Combined_neu'],
      dtype='object')

In [8]:
df = df.drop(labels=['reviews','negative_reviews_processed','positive_reviews_processed','test_negative_review','test_reviews',
                     'test_positive_review'],axis=1)

In [ ]:
df.head(2)

,Additional_Number_of_Scoring,Average_Score,Review_Total_Negative_Word_Counts,Total_Number_of_Reviews,Review_Total_Positive_Word_Counts,Total_Number_of_Reviews_Reviewer_Has_Given,Reviewer_Score,Hotel_Country,review_month,Trip,Stayed nights,yr,reviews_processed,Combined_pos,Combined_neg,Combined_neu
0,241,8.2,138,2500,48,8,7.5,4,8,1,6,2016,bathtub sink flush not working properly even t...,0.155,0.024,0.821
1,493,8.6,23,2865,9,3,5.4,3,12,0,2,2016,bedroom tight small explain hotel built old fa...,0.000,0.000,1.000


In [9]:
X = df.drop(['Combined_pos','Combined_neg','Combined_neu'],axis=1)
y = df['Combined_pos']
y1 = df['Combined_neg']

In [10]:
from sklearn.compose import ColumnTransformer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import OneHotEncoder

In [ ]:
# #sub model1: where target variable is 'Combined_pos'

param ={'dt__max_depth':np.arange(1,100)}
preprocessor = ColumnTransformer(
        transformers=[
       ('tfidf', TfidfVectorizer(analyzer='word', min_df=0,stop_words= stopwords_l), 'reviews_processed')  #TfidfVectorizer accepts column name only between quotes       ('category', OneHotEncoder(), ['categorical_ft']),
          ])
# preprocessor.fit_transform(df)
pipeline = Pipeline([('preprocessor', preprocessor),
                   ('sc',StandardScaler(with_mean=False)),
                   ('dt', DecisionTreeRegressor()),
])
grid = GridSearchCV(pipeline, param_grid= param,cv=3,scoring='neg_mean_squared_error')
grid.fit(X,y)
grid.best_params_
# grid.get_params().keys()

{'dt__max_depth': 94}

In [ ]:
grid.get_params()['estimator__dt__max_depth']

In [ ]:
preprocessor = ColumnTransformer(
        transformers=[
       ('tfidf', TfidfVectorizer( analyzer='word', min_df=0,stop_words= stopwords_l), 'reviews_processed')  #TfidfVectorizer accepts column name only between quotes       ('category', OneHotEncoder(), ['categorical_ft']),
          ])
# preprocessor.fit_transform(df)
full_model1 = Pipeline([('preprocessor', preprocessor),
                   ('sc',StandardScaler(with_mean=False)),
                   ('dt', DecisionTreeRegressor(max_depth=94)),
])
full_model1.fit(X,y)
scores = model_selection.cross_val_score(full_model1, X, y, cv= 3, scoring='neg_mean_squared_error')
rmse = np.sqrt(np.abs(scores))
print('bias_error',np.mean(rmse))
print('var_error',np.std(rmse,ddof=1))

bias_error 0.1477644169567155
var_error 0.000673464078605832


In [ ]:
# #sub model2: where target variable is 'Combined_neg'

param ={'dt__max_depth':np.arange(1,100)}
preprocessor = ColumnTransformer(
        transformers=[
       ('tfidf', TfidfVectorizer(analyzer='word', min_df=0,stop_words= stopwords_l), 'reviews_processed')  #TfidfVectorizer accepts column name only between quotes       ('category', OneHotEncoder(), ['categorical_ft']),
          ])
# preprocessor.fit_transform(df)
pipeline = Pipeline([('preprocessor', preprocessor),
                   ('sc',StandardScaler(with_mean=False)),
                   ('dt', DecisionTreeRegressor()),
])
grid = GridSearchCV(pipeline, param_grid= param,cv=3,scoring='neg_mean_squared_error')
grid.fit(X,y1)
grid.best_params_
# grid.get_params().keys()

{'dt__max_depth': 22}

In [ ]:
preprocessor = ColumnTransformer(
        transformers=[
       ('tfidf', TfidfVectorizer( analyzer='word', min_df=0,stop_words= stopwords_l), 'reviews_processed')  #TfidfVectorizer accepts column name only between quotes       ('category', OneHotEncoder(), ['categorical_ft']),
          ])
# preprocessor.fit_transform(df)
full_model2 = Pipeline([('preprocessor', preprocessor),
                   ('sc',StandardScaler(with_mean=False)),
                   ('dt', DecisionTreeRegressor(max_depth=22)),
])
full_model2.fit(X,y1)
scores = model_selection.cross_val_score(full_model2, X, y1, cv= 3, scoring='neg_mean_squared_error')
rmse = np.sqrt(np.abs(scores))
print('bias_error',np.mean(rmse))
print('var_error',np.std(rmse,ddof=1))

bias_error 0.09809367470408654
var_error 0.0009844363052997271


In [ ]:
#Naive Baise model:
from sklearn.naive_bayes import GaussianNB, MultinomialNB,BernoulliNB
from sklearn.preprocessing import MinMaxScaler,MaxAbsScaler

In [ ]:
#Sub model1 where y='Combined_pos'

# preprocessor = ColumnTransformer(
#         transformers=[
#        ('tfidf', TfidfVectorizer( analyzer='word', min_df=0,stop_words= stopwords_l), 'reviews_processed')  #TfidfVectorizer accepts column name only between quotes       ('category', OneHotEncoder(), ['categorical_ft']),
#           ])
# # preprocessor.fit_transform(df)
# NB_model1 = Pipeline([('preprocessor', preprocessor),
#                    ('sc',StandardScaler(with_mean=False)),
#                   ('nb', MultinomialNB()),
# ])
# NB_model1.fit(X,y)
# scores = model_selection.cross_val_score(NB_model1, X, y, cv= 3, scoring='neg_mean_squared_error')
# rmse = np.sqrt(np.abs(scores))
# print('bias_error',np.mean(rmse))
# print('var_error',np.std(rmse,ddof=1))

In [ ]:
# RandomForest
# tf = TfidfVectorizer(,)

In [ ]:
vectorizer = TfidfVectorizer(analyzer='word', min_df=0,stop_words=stopwords_l)

X_tfidf = vectorizer.fit_transform(X['reviews_processed'])#.toarray() # remember to use the original X_train set
df_tfidf = pd.DataFrame(X_tfidf.toarray())
df_tfidf[['Additional_Number_of_Scoring','Average_Score','Review_Total_Negative_Word_Counts','Total_Number_of_Reviews',
       'Review_Total_Positive_Word_Counts',
       'Total_Number_of_Reviews_Reviewer_Has_Given', 'Reviewer_Score',
       'Hotel_Country', 'review_month', 'Trip', 'Stayed nights', 'yr']] = X[['Additional_Number_of_Scoring','Average_Score',
                                                                             'Review_Total_Negative_Word_Counts','Total_Number_of_Reviews',
                                                                      'Review_Total_Positive_Word_Counts',
                                                                    'Total_Number_of_Reviews_Reviewer_Has_Given', 'Reviewer_Score',
                                                                    'Hotel_Country', 'review_month', 'Trip', 'Stayed nights', 'yr']]

In [ ]:
df_tfidf.shape

(49677, 20697)

In [ ]:
# df_tfidf.to_pickle("/content/drive/My Drive/capstone/df_tfidf.pkl")

In [ ]:
# pkl of tfidf and other numerical features
df_tfidf.to_csv("/content/drive/My Drive/capstone/df_tfidf.csv")

In [ ]:
X_tf =  pd.read_csv("/content/drive/My Drive/capstone/df_tfidf.csv")

In [ ]:
# x_tfidf = pd.concat([X,pd.DataFrame(X_tfidf.toarray())],axis=1)

In [ ]:
rf_be=[]
rf_ve=[]
# param ={'dt__max_depth':np.arange(1,100)}
preprocessor = ColumnTransformer(
        transformers=[
       ('tfidf', TfidfVectorizer(analyzer='word', min_df=0,stop_words= stopwords_l), 'reviews_processed')  #TfidfVectorizer accepts column name only between quotes       ('category', OneHotEncoder(), ['categorical_ft']),
          ])
# preprocessor.fit_transform(df)
pipeline = Pipeline([('preprocessor', preprocessor),
                   ('sc',StandardScaler(with_mean=False)),
                   ('rf', RandomForestRegressor(n_estimators=n)),
])
# pipeline.named_steps['preprocessor'].fit_transform(X['reviews_processed'])
for n in np.arange(1,2):
  pipeline.fit(X,y)
  # pipeline.named_steps['preprocessor'].fit_transform(X['reviews_processed'])
  rf = pipeline.named_steps['rf']
  # kf=model_selection.KFold(shuffle=True,n_splits=3,random_state=0)
  score=model_selection.cross_val_score(rf,X,y,cv=3,scoring='f1_weighted')
  rf_be.append(np.mean(score)),rf_ve.append(np.std(score,ddof=1))

IndexError: ignored

In [ ]:
rf_be=[]
rf_ve=[]
for n in np.arange(1,100):
  rf = RandomForestClassifier(n_estimators=n,criterion='entropy',random_state=0)
  kf=model_selection.KFold(shuffle=True,n_splits=3,random_state=0)
  score=model_selection.cross_val_score(rf,X_std,Y,cv=kf,scoring='f1_weighted')
  rf_be.append(np.mean(1-score)),rf_ve.append(np.std(score,ddof=1))

In [ ]:
rf_be=[]
rf_ve=[]

preprocessor = ColumnTransformer(
        transformers=[
       ('tfidf', TfidfVectorizer( analyzer='word', min_df=0,stop_words= stopwords_l), 'reviews_processed')  #TfidfVectorizer accepts column name only between quotes       ('category', OneHotEncoder(), ['categorical_ft']),
          ])
for n in np.arange(1,10):
#   rf = RandomForestRegressor(n_estimators=n,criterion='entropy')
    rf1 = Pipeline([('preprocessor', preprocessor),
                   ('sc',StandardScaler(with_mean=False)),
                  ('nb', RandomForestRegressor(n_estimators = n)),
    ])
    rf1.fit(X,y)
    score=model_selection.cross_val_score(rf1,X,y,cv= 3,scoring='neg_mean_squared_error')
    rf_be.append(np.mean(score)),rf_ve.append(np.std(score,ddof=1))

KeyboardInterrupt: ignored

In [ ]:



 preprocessor = ColumnTransformer(
        transformers=[
       ('tfidf', TfidfVectorizer( analyzer='word', min_df=0,stop_words= stopwords_l), 'reviews_processed')  #TfidfVectorizer accepts column name only between quotes       ('category', OneHotEncoder(), ['categorical_ft']),
          ])
# preprocessor.fit_transform(df)
NB_model1 = Pipeline([('preprocessor', preprocessor),
                   ('sc',StandardScaler(with_mean=False)),
                  ('nb', RandomForestRegressor(criterion='entropy')),
])
NB_model1.fit(X,y)
scores = model_selection.cross_val_score(NB_model1, X, y, cv= 3, scoring='neg_mean_squared_error')
rmse = np.sqrt(np.abs(scores))
print('bias_error',np.mean(rmse))
print('var_error',np.std(rmse,ddof=1))

In [ ]:
from sklearn.pipeline import Pipeline
from mlxtend.feature_selection import ColumnSelector
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.naive_bayes import BernoulliNB

In [ ]:
X.head(2)

,Additional_Number_of_Scoring,Average_Score,Review_Total_Negative_Word_Counts,Total_Number_of_Reviews,Review_Total_Positive_Word_Counts,Total_Number_of_Reviews_Reviewer_Has_Given,Reviewer_Score,Hotel_Country,review_month,Trip,Stayed nights,yr,reviews_processed,Combined_pos,Combined_neg,Combined_neu
0,241,8.2,138,2500,48,8,7.5,4,8,1,6,2016,bathtub sink flush not working properly even t...,0.155,0.024,0.821
1,493,8.6,23,2865,9,3,5.4,3,12,0,2,2016,bedroom tight small explain hotel built old fa...,0.000,0.000,1.000


In [ ]:
#tfidf 
tfidf = TfidfVectorizer(analyzer='word', min_df=0,stop_words= stopwords_l,ngram_range=(1,2))
# corpus_tfidf = tfidf.fit_transform(df['reviews_processed']).toarray()

In [ ]:
pd.DataFrame(corpus_tfidf).shape

(49677, 20685)